In [1]:
import tensorflow as tf
import os
import sys
from tensorflow import keras
import sklearn
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_score, ConfusionMatrixDisplay
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cv2 import Canny
from skimage.morphology import dilation, erosion, white_tophat, skeletonize
from skimage.morphology import disk 
from skimage import transform
from sklearn.svm import SVC

class_names = ['MALIGNANTMASSES', 'CALC', 'BENIGNMASSES', 'NORM']
class_names_label = {class_names:i for i, class_names in enumerate(class_names)}
print(class_names_label)

IMAGE_SIZE = (224, 224, 3)

def load_data():
    main_directory = r"C:\Users\hp\Documents\CursoML\processed\todas"
    
    output = []
    images = []
    labels = []
         
    for folder in os.listdir(main_directory):
        print(folder)
        label = list(class_names_label.values())[int(folder)-1]

        for file in os.listdir(os.path.join(main_directory, folder)):
            img_path = os.path.join(os.path.join(main_directory, folder), file)

            image = io.imread(img_path, as_gray=True, plugin='matplotlib')
            image = transform.resize(image, IMAGE_SIZE)

            images.append(image)
            labels.append(label)

    images = np.array(images, dtype='float32')
    labels = np.array(labels, dtype='int32')

    output.append(images)
    output.append(labels)
        
    return output

{'MALIGNANTMASSES': 0, 'CALC': 1, 'BENIGNMASSES': 2, 'NORM': 3}


In [2]:
total_images, total_labels = load_data() 

1
2
3
4


In [3]:
train_X, test_X, train_Y, test_Y = train_test_split(total_images, total_labels, test_size=0.4)
print('Datos de aprendizaje: ', train_X.shape, train_Y.shape)
print('Datos de prueba: ', test_X.shape, test_Y.shape)

Datos de aprendizaje:  (683, 224, 224, 3) (683,)
Datos de prueba:  (456, 224, 224, 3) (456,)


In [4]:
import keras
#from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import batch_normalization
from keras.layers.advanced_activations import LeakyReLU

#CREAR EL MODELO

cancer_model = tf.keras.models.Sequential() #pila de capas

cancer_model.add(tf.keras.layers.Conv2D(128, kernel_size=(5,5), activation='linear', padding='same', input_shape=(224,224,3))) #capa de convolucion
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.MaxPooling2D((2, 2),padding='same')) #agrupacion
cancer_model.add(tf.keras.layers.Conv2D(128, kernel_size=(5,5), activation = 'relu'))
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
cancer_model.add(tf.keras.layers.Conv2D(128, (5, 5), activation='linear', padding='same'))
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
cancer_model.add(tf.keras.layers.Flatten())
cancer_model.add(tf.keras.layers.Dense(128, activation='linear'))
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.Dense(4, activation='softmax')) #capa softmax para clasificacion

cancer_model.summary()

cancer_model.compile(optimizer="adam",
                     loss=keras.losses.sparse_categorical_crossentropy,
                     metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 128)     9728      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 128)     409728    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 108, 108, 128)     0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                    

In [ ]:
epochs = 4
batch_size = 64 #numero de muestras por actualizacion 
cancer_train = cancer_model.fit(train_X, train_Y, batch_size=batch_size, epochs=epochs)

Epoch 1/4
